<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/News_group/New_group_classification_base_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/Data/news_group"

/content/drive/MyDrive/Data/news_group


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

## Clean train and test data

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train.head()

,id,text,target
0,0,"\nThey were, and even if Washington might cons...",10
1,1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,2,\n\n\nNot to worry. The Masons have been demo...,19
3,3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,4,Help: I am running some sample problems from O...,5


In [ ]:
def checking_missing_col(dataframe):
  missing_col = []
  for col in dataframe.columns:
    missing_values = sum(dataframe[col].isna())
    is_missing = True if missing_values>=1 else False
    if is_missing:
      print(f"결측치가 있는ㄴ 컬럼은 {col}입니다.")
      print(f"해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.")
  if missing_col == []:
    print("결측치가 존재하지 않습니다.")
  return missing_col

missing_col = checking_missing_col(train)

결측치가 존재하지 않습니다.


In [ ]:
import re
# 숫자와 영문 빼고 모두 제거
def clean_text(texts):
  corpus = []
  for i in range(0, len(texts)):

    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
    review = re.sub(r'\s+', ' ', review) #extra space 제거
    review = re.sub(r'<[^>]+>','',review) #Html tags 제거
    review = re.sub(r'\s+', ' ', review) #spaces 제거
    review = re.sub(r"^\s+", '', review) #space from start 제거
    review = re.sub(r'\s+$', '', review) #space from the end 제거
    review = re.sub(r'_', ' ', review) #space from the end 제거
    corpus.append(review)

  return corpus

In [ ]:
temp = clean_text(train['text']) #메소드 적용
train['text'] = temp
train.head()

,id,text,target
0,0,They were and even if Washington might conside...,10
1,1,We run SpaceNews Views on our STAREACH BBS a l...,14
2,2,Not to worry The Masons have been demonized an...,19
3,3,Only Brendan McKay or maybe ARF would come to ...,17
4,4,Help I am running some sample problems from OR...,5


In [ ]:
temp =clean_text(test['text'])
test['text'] = temp
test.head()

,id,text
0,0,The VLIDE Adapter can be much faster then the ...
1,1,Yeah In a fire that reportedly burned hotter t...
2,2,Judge I grant you immunity from whatever may b...
3,3,I too put a corbin seat on my Hawk I got the s...
4,4,Do I ever After 2 years of having health probl...


use the cleaned train and test data

In [ ]:
!pip install transformers[torch] datasets evaluate

In [ ]:
from sklearn.model_selection import train_test_split

from transformers import (AutoTokenizer,
                          AdamW,
                          AutoModelForSequenceClassification)

from datasets import load_dataset

import torch, gc
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn

import evaluate

from tqdm.notebook import tqdm

In [ ]:
def seed_everything(seed: int = 42):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed= 42
seed_everything(seed)

In [ ]:
target_list = sorted(list(set(train['target'])))
print(len(target_list))
target_list

20


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
# MODEL_NAME = "bert-base-uncased"
# MODEL_NAME = 'google/electra-base-discriminator'
MODEL_NAME = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# train_set, valid_set =train_test_split(train, test_size=0.2, stratify=train.target.values)

In [ ]:
# # split train valid dataset
# train_set = train_set.sample(n=2000)
# valid_set = valid_set.sample(n=500)

In [ ]:
# train_set = train_set.reset_index(drop=True)
# valid_set = valid_set.reset_index(drop=True)

In [ ]:
max_len = 512

class CustomDataset(Dataset):

  def __init__(self, dataset, train_mode = True):
    self.dataset = dataset
    self.train_mode = train_mode

  def __getitem__(self, index):
    text = self.dataset.loc[index, 'text']

    encoded_data = tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = max_len,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        return_tensors ='pt'
        )
    input_ids = encoded_data['input_ids'][0]
    attention_masks = encoded_data['attention_mask'][0]

    if self.train_mode:
      labels = torch.tensor(self.dataset.loc[index, 'target'])
      return input_ids, attention_masks, labels
    return input_ids, attention_masks

  def __len__(self):
    return len(self.dataset)

In [ ]:
# train_dataset = CustomDataset(train_set, train_mode=True)
# valid_dataset = CustomDataset(valid_set, train_mode=True)

In [ ]:
BATCH_SIZE = 16

# trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# validloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def compute_metrics(pred, label):
  return sum(pred==label)/len(label)

In [ ]:
id2label = {0 : 'alt.atheism', 1: 'comp.graphics',2 : 'comp.os.ms-windows.misc',
           3 : 'comp.sys.ibm.pc.hardware', 4 : 'comp.sys.mac.hardware',
           5 : 'comp.windows.x', 6 : 'misc.forsale', 7: 'rec.autos',
           8 : 'rec.motorcycles', 9 : 'rec.sport.baseball', 10 : 'rec.sport.hockey',
           11 : 'sci.crypt', 12 : 'sci.electronics', 13 : 'sci.med',
           14 : 'sci.space', 15 : 'soc.religion.christian', 16 : 'talk.politics.guns',
           17 : 'talk.politics.mideast', 18 : 'talk.politics.misc', 19 : 'talk.religion.misc'}


label2id = {'alt.atheism': 0, 'comp.graphics': 1, 'comp.os.ms-windows.misc': 2,
            'comp.sys.ibm.pc.hardware': 3, 'comp.sys.mac.hardware': 4, 'comp.windows.x': 5,
            'misc.forsale': 6, 'rec.autos': 7, 'rec.motorcycles': 8, 'rec.sport.baseball': 9,
            'rec.sport.hockey': 10, 'sci.crypt': 11, 'sci.electronics': 12, 'sci.med': 13,
            'sci.space': 14, 'soc.religion.christian': 15,'talk.politics.guns': 16,
            'talk.politics.mideast': 17, 'talk.politics.misc': 18, 'talk.religion.misc': 19}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                           num_labels=len(target_list),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# class Trainer:
#   def __init__(self, trainloader, validloader, model, optimizer, criterion, device):
#     self.trainloader = trainloader
#     self.validloader = validloader
#     self.model = model
#     self.optimizer = optimizer
#     self.criterion = criterion
#     self.device = device


#   def train(self, epochs=1):
#     gc.collect()
#     self.model.train()
#     best_acc = None
#     for epoch in range(1, epochs+1):
#       running_loss = 0.0
#       for input_ids, attention_mask, y in tqdm(self.trainloader):
#         input_ids = input_ids.to(self.device)
#         attention_mask = attention_mask.to(self.device)
#         y = y.to(self.device)

#         self.optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask)[0]
#         loss = self.criterion(outputs, y)
#         loss.backward()
#         self.optimizer.step()
#         running_loss += loss.item()
#       train_loss = running_loss/len(self.trainloader)
#       valid_loss , predictions, labels = self.validate()
#       accuracy = compute_metrics(predictions, labels)
#       tqdm.write('Epoch:{:1d}, Train Loss: {:.3f}, Validation Loss: {:.3f}, Accuracy: {:.3f}'.format(
#           epoch, train_loss, valid_loss, accuracy))
#       if best_acc is None:
#         best_acc = accuracy
#         torch.save(model.state_dict(), f'./Roberta.model')
#       elif best_acc <= accuracy:
#         best_acc = accuracy
#         torch.save(model.state_dict(), f'./Roberta.model')
#       else:
#         break

#     return accuracy

#   def validate(self):
#     self.model.eval()
#     predictions, labels = [], []
#     with torch.no_grad():
#       running_loss = 0.0
#       for input_ids, attention_mask, y in tqdm(self.validloader):
#         input_ids = input_ids.to(self.device)
#         attention_mask = attention_mask.to(self.device)
#         y = y.to(self.device)

#         outputs = model(input_ids, attention_mask)[0]
#         loss = self.criterion(outputs, y).to(self.device)
#         running_loss += loss.item()

#         pred = outputs.detach().cpu().numpy().argmax(1)
#         label = y.detach().cpu().numpy()
#         predictions.append(pred)
#         labels.append(label)

#       valid_loss = running_loss/len(self.validloader)
#       predictions =  np.concatenate(predictions, axis=0)
#       labels = np.concatenate(labels, axis=0)
#       return valid_loss, predictions, labels

In [ ]:
Learning_rate =  2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion =  nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr = Learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# trainer = Trainer(trainloader= trainloader,
#               validloader = validloader,
#               model = model,
#               optimizer = optimizer,
#               criterion = criterion,
#               device = device)
# trainer.train(epochs = 10)

  0%|          | 0/462 [00:00<?, ?it/s]

  0%|          | 0/116 [00:00<?, ?it/s]

Epoch:1, Train Loss: 1.872, Validation Loss: 1.206, Accuracy: 0.610


  0%|          | 0/462 [00:00<?, ?it/s]

  0%|          | 0/116 [00:00<?, ?it/s]

...

Epoch:3, Train Loss: 0.653, Validation Loss: 0.961, Accuracy: 0.718


  0%|          | 0/462 [00:00<?, ?it/s]

  0%|          | 0/116 [00:00<?, ?it/s]

Epoch:4, Train Loss: 0.354, Validation Loss: 1.039, Accuracy: 0.702


0.7022198159177044

**Bert** "bert-base-uncased"
- Learning_rate =  2e-5
- Epoch:3, Train Loss: 0.451, Validation Loss: 0.930, Accuracy: 0.733

**Electra** 'google/electra-base-discriminator'
- Learning_rate =  2e-5
- Epoch:4, Train Loss: 0.517, Validation Loss: 0.988, Accuracy: 0.720


**Roberta** 'xlm-roberta-base'
- Learning_rate =  2e-5
- Epoch:3, Train Loss: 0.653, Validation Loss: 0.961, Accuracy: 0.718


In [ ]:
def predict(model_load, testloader):
  model.load_state_dict(torch.load(model_load, map_location=device))
  model.eval()
  probs = None
  with torch.no_grad():
    for input_ids, attention_mask in tqdm(testloader):
      input_ids = input_ids.to(device)
      attention_mask = attention_mask.to(device)

      outputs = model(input_ids, attention_mask)[0]
      outputs = outputs.cpu().numpy()
      if probs is None:
        probs = outputs
      else:
        probs = np.concatenate([probs, outputs])

    _, test_preds = torch.max(torch.tensor(probs), dim=1)
  return test_preds

In [ ]:
test_set = test.reset_index(drop=True)
test_set.head()

,id,text
0,0,The VLIDE Adapter can be much faster then the ...
1,1,Yeah In a fire that reportedly burned hotter t...
2,2,Judge I grant you immunity from whatever may b...
3,3,I too put a corbin seat on my Hawk I got the s...
4,4,Do I ever After 2 years of having health probl...


In [ ]:
test_dataset = CustomDataset(test_set, train_mode=False)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model_load = './Roberta.model'
preds = predict(model_load, testloader)

  0%|          | 0/578 [00:00<?, ?it/s]

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
submission['target'] = preds
submission.head()

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13


In [ ]:
submission.to_csv('./submission_roberta.csv', index=False)